## 我们建立一个名为sources的变量，它包含所有的原始数据文件：

In [1]:
HAM = 0
SPAM = 1
datadir = '../data/chapter7'
sources = [
    ('beck-s.tar.gz', HAM),
    ('farmer-d.tar.gz', HAM),
    ('kaminski-v.tar.gz', HAM),
    ('kitchen-l.tar.gz', HAM),
    ('lokay-m.tar.gz', HAM),
    ('williams-w3.tar.gz', HAM),
    ('BG.tar.gz', SPAM),
    ('GP.tar.gz', SPAM),
    ('SH.tar.gz', SPAM)
]

## 把这些文件提取到子目录。

In [2]:
def extract_tar(datafile, extractdir):
    try:
        import tarfile
    except ImportError:
        raise ImportError("You do not have tarfile installed. "
                          "Try unzipping the file outside of Python.")

    tar = tarfile.open(datafile)
    tar.extractall(path=extractdir)
    tar.close()
    print("%s successfully extracted to %s" % (datafile, extractdir))

for source, _ in sources:
    datafile = '%s/%s' % (datadir, source)
    extract_tar(datafile, datadir)

../data/chapter7/beck-s.tar.gz successfully extracted to ../data/chapter7
../data/chapter7/farmer-d.tar.gz successfully extracted to ../data/chapter7
../data/chapter7/kaminski-v.tar.gz successfully extracted to ../data/chapter7
../data/chapter7/kitchen-l.tar.gz successfully extracted to ../data/chapter7
../data/chapter7/lokay-m.tar.gz successfully extracted to ../data/chapter7
../data/chapter7/williams-w3.tar.gz successfully extracted to ../data/chapter7
../data/chapter7/BG.tar.gz successfully extracted to ../data/chapter7
../data/chapter7/GP.tar.gz successfully extracted to ../data/chapter7
../data/chapter7/SH.tar.gz successfully extracted to ../data/chapter7


read_single_file(filename)：这个函数提取来自于filename文件的相关内容。

read_files(path)：这个函数提取来自于path特定目录中的所有文件的相关内容。

In [3]:
import os

def read_single_file(filename):
    past_header, lines = False, []
    if os.path.isfile(filename):
        # 因为一些邮件没有使用统一码，这条语句试图正确解码文件。
        f = open(filename, encoding="latin-1")
        for line in f:
            if past_header:
                lines.append(line)
            elif line == '\n':
                past_header = True
        f.close()
    content = '\n'.join(lines)
    return filename, content

def read_files(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(root, filename)
            yield read_single_file(filepath)

## 使用pandas建立一个数据矩阵：

In [4]:
import pandas as pd

def build_data_frame(extractdir, classification):
    rows = []
    index = []
    for file_name, text in read_files(extractdir):
        rows.append({'text': text, 'class': classification})
        index.append(file_name)

    data_frame = pd.DataFrame(rows, index=index)
    return data_frame

data = pd.DataFrame({'text': [], 'class': []})
for source, classification in sources:
    extractdir = '%s/%s' % (datadir, source[:-7])
    data = data.append(build_data_frame(extractdir, classification),sort=True)

## 数据预处理

在文本特征编码时，scikit-learn提供了几个选项，文本数据编码的一种最简单方式是单词计数；对于每个短语，计算其中每个单词出现的次数。在scikit-learn中，使用CountVectorizer很容易实现这个任务：

In [5]:
from sklearn import feature_extraction
counts = feature_extraction.text.CountVectorizer()
X = counts.fit_transform(data['text'].values)
X.shape

(52076, 643270)

结果是一个大的矩阵，它告诉我们一共收集了52 076封电子邮件，总共包含643270个不同的单词。scikit-learn很明智地把数据保存在一个稀疏矩阵中：

In [6]:
X

<52076x643270 sparse matrix of type '<class 'numpy.int64'>'
	with 8607632 stored elements in Compressed Sparse Row format>

要建立目标标签（y）的向量，我们需要访问pandas DataFrame中的数据。这通过把DataFrame看成一个字典来实现，其中values属性将允许我们访问底层的NumPy数组：

In [7]:
y = data['class'].values

## 训练一个普通贝叶斯分类器

In [8]:
# 使用scikit-learn把数据拆分成训练集和测试集（让我们保留所有数据点的20%作为测试集）：
from sklearn import model_selection as ms
X_train, X_test, y_train, y_test = ms.train_test_split(
    X, y, test_size=0.2, random_state=42
)
# 用OpenCV实例化一个新的普通贝叶斯分类器：
import cv2
model_norm = cv2.ml.NormalBayesClassifier_create()

可是，OpenCV并不了解稀疏矩阵（至少它的Python接口不了解）。如果我们像之前那样把X_train和y_train传递给train函数，OpenCV会抱怨数据矩阵不是一个NumPy数组。但是把稀疏矩阵转换成一个常规的NumPy数组可能会耗尽内存。因此，一种可能的解决方法是，只在数据点（1000）和特征（300）的一个子集上训练OpenCV分类器：

In [9]:
import numpy as np
X_train_small = X_train[:1000, :300].toarray().astype(np.float32)
y_train_small = y_train[:1000].reshape((1000,-1)).astype(np.int32)

model_norm.train(X_train_small, cv2.ml.ROW_SAMPLE, y_train_small)

True

## 在完整数据集上训练

使用scikit-learn的朴素贝叶斯分类器，因为它知道如何处理稀疏矩阵。实际上，如果你不留意，把X_train当成之前的每个NumPy数组，那么你很可能没有发现有什么区别：

In [10]:
from sklearn import model_selection as ms
X_train, X_test, y_train, y_test = ms.train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 使用naive_bayes模块的MultinomialNB，这是最适合处理类别数据（如单词计数）的朴素贝叶斯分类器。
from sklearn import naive_bayes
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)

MultinomialNB()

In [11]:
model_naive.score(X_train, y_train), model_naive.score(X_test, y_test)

(0.9508641382621219, 0.9442204301075269)

## 使用n-grams提升结果

到目前为止，我们依赖的是词袋：我们只是简单地把一封电子邮件中的每一个单词都放入一个袋子，并计算出现的次数。可是，在真正的邮件中，单词在邮件中出现的顺序可以携带很多有用信息！

这正是n-grams计数试图传达的信息。你可以把一个n-grams想象成是长度为n的一个短语。例如，短语Statistics has its moments包含以下1-grams: Statistics、has、its和moments；包含以下2-grams: Statistics has、has its和itsmoments；还有2个3-grams（Statistics has its和has its moments）以及唯一1个4-grams。

我们可以为n指定一个范围，让CountVectorizer把任意序列的n-grams包含到特征矩阵中：

In [12]:
counts = feature_extraction.text.CountVectorizer(
    ngram_range=(1, 2)
)
X = counts.fit_transform(data['text'].values)

from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)

MultinomialNB()

In [13]:
model_naive.score(X_test, y_test)

0.9706221198156681

可是，n-grams计数并不完美。它们的缺点是不可能加权长文档（因为有可能形成更多的n-grams组合）。为了避免这个问题，我们可以使用相对次数，而不是使用简单的出现次数。

## 使用TF-IDF提升结果

它称为词率–逆文档频率（Term Frequency-Inverse Document Frequency，TF-IDF），IF-IDF所实现的基本上是通过度量单词在整个数据集中出现的频率来加权单词的数量。这种方法的一个有用的副作用是IDF部分——单词频率的逆。这就保证频繁出现的单词，如and、the和but在分类中只占很小的权重。

在我们现有的特征矩阵X上，通过调用fit_transform，把TF-IDF应用到特征矩阵：

In [14]:
tfidf = feature_extraction.text.TfidfTransformer()
X_new = tfidf.fit_transform(X)

# 如果训练–测试的拆分发生了变化，那么整体准确率可能会发生变化：
X_train, X_test, y_train, y_test = ms.train_test_split(
    X_new, y, test_size=0.2, random_state=42
)

# 再次训练分类器并对其进行评分。
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)
model_naive.score(X_test, y_test)

0.9908794162826421

为了证明分类器的确很棒，我们可以检查混淆矩阵。这是对于每一个类，显示有多少数据样本被错误地归类为属于不同的类的一个矩阵。其对角线元素告诉我们类i有多少样本是正确分类为属于类i的；非对角线元素表示分类错误：

In [15]:
from sklearn import metrics
metrics.confusion_matrix(y_test, model_naive.predict(X_test))

array([[3746,   84],
       [  11, 6575]], dtype=int64)

这告诉我们3746个0类分类正确，6565个1类分类正确。我们把84个0类样本混淆为属于1类，11个1类样本混淆为属于0类。